From paper:
https://github.com/maitreyeeT/Seq2SeqDialStruct/blob/431419204a4156d633d0d744657fc0cbeef08ce6/Seq2SeqVAE_dialogueStructuring/vae_LSTM.py#L39

Look for tutorials on VAE. preferably pytorch? (or not, i don't think i used it yet)

In [ ]:
import keras
# from Seq2SeqVAE_dialogueStructuring.variational_layer import VariationalLayer
l = keras.layers
m = keras.models
K = keras.backend
opt = keras.optimizers
losses = keras.losses
cbk = keras.callbacks
reg = keras.regularizers


class LSTMVAutoencoder:

    def __init__(self, lr, window_size, sequence_size, dropout=0.):
        self.lr = lr
        self.window_size = window_size
        self.sequences_length = sequence_size
       # self.input_convolution_size = 3
       # self.conv_2d_filters = []  # Num filter of the convolution layers
       # self.conv_2d_kernels = []  # Kernel sizes of the convolution layers

        self.dropout = dropout
        self.regul = 1e-4
        self.latent_dim = 256
        self.build()

    def get_attention(self,name, output_size, dropout=0., axis=-2):
        model_att = m.Sequential(name='attention_' + str(name))  # (?, n, k, x)
        model_att.add(l.Dense(1, activation=K.exp))  # (?, n, k, 1)
        model_att.add(l.Lambda(lambda x: x / (K.sum(x, axis=axis, keepdims=True) + \
                                              K.epsilon()),
                               name=name + 'attention'))  # (?, n, k, 1)

        m_att = m.Sequential()
        m_att.add(l.Lambda(lambda x: K.sum(x * model_att(x), axis=axis)))
        m_att.add(l.Dense(output_size, activation='linear'))
        return m_att

    def make_networks(self):
        attention = self.get_attention('xylo',10)
        #from w2vec_utils import train_word2vec, data_transform
        #data_embed = data_transform()
        #w2vmodel, embed_wts = train_word2vec(data_embed)
        encoder = m.Sequential(name='encoder')
        #encoder.add(l.Embedding(output_dim=300, weights=[embed_wts],input_dim=embed_wts.shape[0],
        #                        input_shape=(self.sequences_length,2)))
        #encoder.add(l.Lambda(lambda x: K.mean(x, axis=-2)))
        encoder.add(l.LSTM(10, return_sequences='True'))
        encoder.add(l.LSTM(10, return_sequences='True'))

        encoder.add(attention)
        #encoder.add(l.Dense(5, activity_regularizer=reg.l2(1e-3)))

        self.encoder = encoder

        decoder = m.Sequential(name='decoder')
        decoder.add(l.RepeatVector(self.sequences_length))
        decoder.add(l.LSTM(10, return_sequences=True))
        decoder.add(l.LSTM(self.window_size, activation='sigmoid'
                           , return_sequences=True))

        self.decoder = decoder

    def build(self):

        seq_input = l.Input(shape=(self.sequences_length, self.window_size),
                            dtype='float32')  # (?, n, k)

        self.make_networks()

        latent_in = self.encoder(seq_input)
    #     latent_in = VariationalLayer(10, kl_gain=1e-3)(latent_in) <<<< variational layer. commented since i don't yet hve the code for this

        decoder_out = self.decoder(latent_in)

        model = m.Model(inputs=[seq_input], outputs=[decoder_out])

        model_latent = m.Model(inputs=[seq_input], outputs=[latent_in])

        model.compile(opt.Adam(self.lr), loss=[lambda x, y: losses.binary_crossentropy(x, y)])

        model_latent.compile(opt.RMSprop(1e-3), loss='binary_crossentropy')
        self.model = model
        self.model_latent = model_latent

    def fit(self, X, epochs, callbacks=None, verbose=2):

        self.model.fit(x=[X],
                       y=[X],
                       batch_size=30,
                       epochs=epochs,
                       validation_split=.2,
                       callbacks=callbacks,
                       verbose=verbose)

### DL class project on VAE

In [84]:
#from DL class, project 4
# import numpy as np
# import torch
# import torchvision
# import matplotlib.pyplot as plt
# from time import time
# import os
# from torchvision import datasets, transforms
# from torch import optim, nn, unsqueeze
# from torch.utils.data import DataLoader
# from torchvision.transforms import ToTensor, Lambda, Compose
# import torch.nn as nn
# import torch.nn.functional as F
# from sklearn.model_selection import train_test_split
# import torch.nn.functional as F
# import torchvision.utils as vutils
# import torch.nn as nn
# from torch.distributions.log_normal import LogNormal
# from torchvision.utils import save_image

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# class VAE(nn.Module):
#     def __init__(self, D, M):
#         super(VAE, self).__init__()
#         self.D = D
#         self.M = M

#         self.enc1 = nn.Linear(in_features=self.D, out_features=300)
#         self.enc2 = nn.Linear(in_features=300, out_features=self.M*2)

#         self.dec1 = nn.Linear(in_features=self.M, out_features=300)
#         self.dec2 = nn.Linear(in_features=300, out_features=self.D)

#     def reparameterize(self, mu, log_std): 
#         std = torch.exp(log_std)
#         eps = torch.randn_like(std)
#         z = mu + (eps * std)
#         return z

#     def forward(self, x): 
#         # encoder
#         x = self.enc1(x)
#         x = nn.functional.relu(x)
#         x = self.enc2(x).view(-1, 2, self.M)
        
#         # get mean and log-std
#         mu = x[:, 0, :]
#         log_std = x[:, 1, :]
        
#         # reparameterization
#         z = self.reparameterize(mu, log_std)

#         # decoder
#         x_hat = nn.functional.relu(self.dec1(z))
#         x_hat = torch.sigmoid(self.dec2(x_hat))
#         return x_hat, mu, z, log_std

#     def generate(self, z):
#         x_hat = nn.functional.relu(self.dec1(z))
#         x_hat = torch.sigmoid(self.dec2(x_hat))
#         return x_hat

#     def elbo(self, x, x_hat, z, mu, log_std): 
#         # reconstruction error
#         # RE = nn.loss.mse(x, x_hat)

#         RE = F.binary_cross_entropy(x_hat, x)

#         # kl-regularization
#         # We assume here that log_normal is implemented
#         # KL = LogNormal(z, mu, log_std) - LogNormal(z, 0, 1)

#         KL = -0.5 * torch.sum(1 + log_std - mu.pow(2) - log_std.exp())
#         KL /= (784 * x_hat.size(0))

#         # REMEMBER! We maximize ELBO, but optimizers minimize. # Therefore, we need to take the negative sign!
#         return -(RE - KL)


In [85]:
# def fit(model, dataloader, optimizer):
#     model.train()
#     running_loss = 0.0

#     for i, data in enumerate(dataloader):

#         data = data[0].to(device)
#         data = data.view(data.size(0),-1)

#         optimizer.zero_grad()
#         data_hat, mu,z, log_std = model(data)
#         loss = model.elbo(x = data, x_hat = data_hat,z=z, mu = mu, log_std = log_std)
#         running_loss += loss.item()
#         loss.backward()
#         optimizer.step()

#     train_loss = running_loss/len(dataloader.dataset)
#     return train_loss

# def validate(model, valdataloader, epoch):
#     model.eval()
#     running_loss = 0.0
#     with torch.no_grad():
#         for i, data in enumerate(valdataloader):
#             #data, _ = data
#             data = data[0].to(device)
#             data = data.view(data.size(0),-1)
#             data_hat, mu, z, log_std = model(data)
#             loss = model.elbo(data, data_hat,z, mu, log_std)
#             running_loss += loss.item()

#             if (epoch%10 == 0) and (i == len(valiloader)-1):
#                 # save the last 8 samples input and output of every 10th epoch
#                 num_rows = 8
#                 both = torch.cat((data.view(len(data), 1, 28, 28)[:8], 
#                                   data_hat.view(len(data), 1, 28, 28)[:8]))

#                 torchvision.utils.save_image(both.cpu(), f"./output{epoch}.png", nrow=num_rows)
    
#     #val_loss = running_loss/len(valiloader.dataset)
#     return running_loss/len(valiloader.dataset)

# def train(model, trainloader, valiloader, optimizer, epochs):
#     train_loss, val_loss = [], []

#     for epoch in range(epochs):
#         print(f"Epoch {epoch+1} of {epochs}")
#         train_epoch_loss = fit(model, trainloader, optimizer)
#         val_epoch_loss = validate(model, valiloader, epoch)
#         train_loss.append(train_epoch_loss)
#         val_loss.append(val_epoch_loss)
#         print(f"Train Loss: {train_epoch_loss:.4f}")
#         print(f"Val Loss: {val_epoch_loss:.4f}")

#         return model, train_loss, val_loss

# BATCH_SIZE = 64

# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]) 
# # transform = transforms.Compose([transforms.Resize((32, 32)), 
# #                                 transforms.ToTensor(),
# #                                 transforms.Lambda(lambda x: x.repeat(3,1,1)),
# #                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# #load in full train set
# trainsetfull = torchvision.datasets.MNIST(root='./data/mnist', train=True, download=True, transform=transform)
# # type(trainsetfull)

# # data loader for final run 
# trainfullloader = torch.utils.data.DataLoader(trainsetfull, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
# #split the set 
# trainset, valset = torch.utils.data.random_split(trainsetfull, [55000, 5000])
# #load in test set
# testset = torchvision.datasets.MNIST(root='./data/mnist', train=False, transform=transform,download=True)


# # data loader for training
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True, num_workers=2)
# # data loader for validation
# valiloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE,shuffle=True, num_workers=2)
# # data loader for testing
# testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False, num_workers=2)

# #MNIST
# net = VAE(D = 28*28*1, M = 16)
# # net = VAE(D = 32*32*3, M = 16)
# net.to(device)
# optimizer = optim.Adam(net.parameters(), lr = 0.005)
# net, train_loss, val_loss= train(net, trainloader, valiloader, optimizer, 50 )

#values to plot
# val_loss

## One hot vector of DAs from whole dialogues

In [68]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils.vis_utils import plot_model
import pandas as pd
from ngrams import add_special_tok

In [182]:
df = pd.read_csv('./data_TM2/processed/processed_utterances_sentence_DA_labeling.csv', index_col=0)

df, full_DA, unique_ids = add_special_tok(df)

In [186]:
from nltk import ngrams
import pickle

unique = list(df['all_DA'].unique())
one_hot_ubtf = []

#uni
[one_hot_ubtf.append([x]) for x in unique]

#bi
for i in range(len(unique)):
    for j in range(len(unique)):
        one_hot_ubtf.append([unique[i], unique[j]])
        
#tri >>>>>> until here 12,719
for i in range(len(unique)):
    for j in range(len(unique)):
        for k in range(len(unique)):
            one_hot_ubtf.append([unique[i], unique[j], unique[k]])
            
print(len(one_hot_ubtf))
one_hot_ubtf

#################

dict_grams = {}

for e, pat in enumerate(full_DA):
    pat = (' ').join(pat)

    for uni in ngrams(pat.split(), 1):
        uni = list(uni)
        if pat not in dict_grams:
            dict_grams[pat] = [uni]
        else:
            dict_grams[pat].extend([uni])

    for bi in ngrams(pat.split(), 2):
        bi = list(bi)
        if pat not in dict_grams:
            dict_grams[pat] = [bi]
        dict_grams[pat].extend([bi])

    for tri in ngrams(pat.split(), 3):
        tri = list(tri)
        if pat not in dict_grams:
            dict_grams[pat] = [tri]
        dict_grams[pat].extend([tri])

dict_grams

with open('./src/generated_files/dict_all_possible_grams.pkl', 'wb') as fp:
    pickle.dump(one_hot_ubtf, fp)

12719


In [213]:
import itertools

# create matrix size len(one_hot_ubtf) X len(hmm_results). TO GET FULL SIZE MATRIX:
# one_hot_mat = np.zeros((len(dict_grams.keys()), len(one_hot_ubtf))) #format: mat_one_hot[row=dict_grams.keys][column_one_hot]


# create matrix size len(one_hot_ubtf) X len(hmm_results). TO GET SAMPLE:
one_hot_mat = np.zeros((100, len(one_hot_ubtf))) #format: mat_one_hot[row=dict_grams.keys][column_one_hot]

dict_grams_sample = dict(itertools.islice(dict_grams.items(), 100))
# print(dict_grams_sample)

for e, pattern in enumerate(dict_grams_sample.values()):
    for g, pat in enumerate(pattern):
        for f, pos_ngr in enumerate(one_hot_ubtf):
            if pos_ngr == pat:
                one_hot_mat[e][f] = 1


In [214]:
len(one_hot_mat[0])

12719

## LSTM Autoencoders

In [122]:
#only AE with LSTM. from here https://machinelearningmastery.com/lstm-autoencoders/

# lstm autoencoder reconstruct and predict sequence
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# define input sequence
seq_in = array([0,1,2,3,4,0]) # seq of DAs 
# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))
# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

# define encoder
visible = Input(shape=(n_in,1))
encoder = LSTM(10, activation='relu')(visible)
print(encoder)

# define reconstruct decoder
decoder1 = RepeatVector(n_in)(encoder)
decoder1 = LSTM(10, activation='relu', return_sequences=True)(decoder1)
decoder1 = TimeDistributed(Dense(1))(decoder1)

# define predict decoder
decoder2 = RepeatVector(n_out)(encoder)
decoder2 = LSTM(100, activation='relu', return_sequences=True)(decoder2)
decoder2 = TimeDistributed(Dense(1))(decoder2)

# tie it together
model = Model(inputs=visible, outputs=[decoder1, decoder2])
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(seq_in, [seq_in,seq_out], epochs=300, verbose=0)

# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat)

KerasTensor(type_spec=TensorSpec(shape=(None, 10), dtype=tf.float32, name=None), name='lstm_213/strided_slice_3:0', description="created by layer 'lstm_213'")
[array([[[ 0.2881454 ],
        [ 0.8721983 ],
        [ 1.8641942 ],
        [ 3.1861396 ],
        [ 3.9022322 ],
        [-0.01377723]]], dtype=float32), array([[[ 0.9651851 ],
        [ 1.9545611 ],
        [ 3.0658395 ],
        [ 3.9779654 ],
        [-0.00785341]]], dtype=float32)]


## from Keras

In [239]:
# lstm autoencoder reconstruct and predict sequence
#https://keras.io/examples/generative/vae/

from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Flatten, Layer
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class Sampling(Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


latent_dim = 2

n_in = len(one_hot_mat[0])

# def encode(n_in = len(one_hot_mat[0])):
# define encoder
visible = Input(shape=(n_in,1))
x = LSTM(10, activation='relu', return_sequences=True)(visible)
x = LSTM(10, activation='relu', return_sequences=True)(x) 
x = Flatten()(x)
x = Dense(1, activation="relu")(x)

z_mean = Dense(latent_dim, name="z_mean")(x)
z_log_var = Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = Model(visible, [z_mean, z_log_var, z], name="encoder")
print(encoder.summary())
#     return encoder


# def decode(encoder):
latent_inputs = Input(shape=(latent_dim,))
x = keras.layers.Flatten()(x) 
x = Dense(10*n_in, activation="relu")(latent_inputs)
x = layers.Reshape((n_in, 10))(x)
x = LSTM(10, activation='relu', return_sequences=True)(x)
x = LSTM(10, activation='relu', return_sequences=True)(x) #############
decoder = Model(latent_inputs, x, name="decoder")
print(decoder.summary())
#     return decoder


Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_138 (InputLayer)          [(None, 12719, 1)]   0                                            
__________________________________________________________________________________________________
lstm_312 (LSTM)                 (None, 12719, 10)    480         input_138[0][0]                  
__________________________________________________________________________________________________
lstm_313 (LSTM)                 (None, 12719, 10)    840         lstm_312[0][0]                   
__________________________________________________________________________________________________
flatten_55 (Flatten)            (None, 127190)       0           lstm_313[0][0]                   
____________________________________________________________________________________________

In [240]:
# Define the VAE as a Model with a custom train_step

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)))
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {"loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result()}

In [241]:
# Train the VAE
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
print(type(mnist_digits))
print(type(one_hot_mat))

print(one_hot_mat.shape)
print(mnist_digits.shape)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(one_hot_mat, epochs=30, batch_size=128)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(100, 12719)
(70000, 28, 28, 1)
Epoch 1/30


ValueError: in user code:

    /Users/brunaguedes/opt/anaconda3/envs/inteamenv/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/brunaguedes/opt/anaconda3/envs/inteamenv/lib/python3.9/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/brunaguedes/opt/anaconda3/envs/inteamenv/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/brunaguedes/opt/anaconda3/envs/inteamenv/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/brunaguedes/opt/anaconda3/envs/inteamenv/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return

In [166]:
# Display how the latent space clusters different digit classes
def plot_label_clusters(vae, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = vae.encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()

(x_train, y_train), _ = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype("float32") / 255

plot_label_clusters(vae, x_train, y_train)

Variational autoencoders are unsupervised learning methods in the sense that they don't require labels in addition to the data inputs. All that is required for VAE is to define an appropriate likelihood function for your data. Commonly, a reconstruction loss is used, e.g. binary cross-entropy, squared distance, or absolute distance, to compare the actual input to the VAE with the generated output of the VAE. If you want to use VAE for dimensionality reduction or feature extraction, you would feed our data to the encoder network and use its output for downstream tasks.

Keep in mind that VAE is a generative model, thus training encourages the output of the encoder to resemble an isotropic Gaussian distribution. If you don't need to generate data and just want to extract features, you might get better results with a traditional autoencoder without the Kullback-Leibler divergence regularizer that is used in VAE